Flight Price Prediction (EDA and Feature Engineering)

In [135]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#For Data Viz. 
import matplotlib.pyplot as plt
import seaborn as sns 

#For Warnings
import warnings
warnings.filterwarnings('ignore')

#For Options 
pd.set_option('display.max_columns' , None)
pd.set_option('display.max_rows', None)

from sklearn.model_selection import train_test_split 

#For Kaggle data import. 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [88]:

train_df = pd.read_excel('Data_Train.xlsx')
test_df = pd.read_excel('Test_set.xlsx')


In [89]:
train_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [90]:
test_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [91]:
df = train_df.append(test_df)

df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0


In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Date_of_Journey  13354 non-null  object 
 2   Source           13354 non-null  object 
 3   Destination      13354 non-null  object 
 4   Route            13353 non-null  object 
 5   Dep_Time         13354 non-null  object 
 6   Arrival_Time     13354 non-null  object 
 7   Duration         13354 non-null  object 
 8   Total_Stops      13353 non-null  object 
 9   Additional_Info  13354 non-null  object 
 10  Price            10683 non-null  float64
dtypes: float64(1), object(10)
memory usage: 1.2+ MB


In [93]:
def missing_vals(df): 
    for i in df :
        if df[i].isnull().sum() > 0 : 
            print(f"{i} : {df[i].isnull().sum()} missing values out of {len(df[i])}")
    else : 
        print("No missing columns.")
missing_vals(df)


Route : 1 missing values out of 13354
Total_Stops : 1 missing values out of 13354
Price : 2671 missing values out of 13354
No missing columns.


In [94]:
df['Route'] = df['Route'].fillna(df['Route'].mode()[0])
df['Total_Stops'] = df['Total_Stops'].fillna(df['Total_Stops'].mode()[0])

In [95]:
missing_vals(df)

Price : 2671 missing values out of 13354
No missing columns.


Feature Engineering Process. 

In [96]:
df['Date'] = df['Date_of_Journey'].str.split('/').str[0]
df['Month'] = df['Date_of_Journey'].str.split('/').str[1]
df['Year'] =  df['Date_of_Journey'].str.split('/').str[2]

In [97]:
df['Date'] = df['Date'].astype(int)
df['Month'] = df['Month'].astype(int)
df['Year'] = df['Year'].astype(int)

In [98]:
df.drop('Date_of_Journey' , axis = 1 , inplace = True ) 

In [99]:
df['Arrival_Time'] = df['Arrival_Time'].str.split(' ').str[0]

In [100]:
df['Arrival_Hour'] = df['Arrival_Time'].str.split(':').str[0]
df['Arrival_Minutes'] = df['Arrival_Time'].str.split(':').str[1]

In [101]:
df['Arrival_Hour'] = df['Arrival_Hour'].astype(int)
df['Arrival_Minutes'] = df['Arrival_Minutes'].astype(int)


In [102]:
df.drop('Arrival_Time',axis = 1 , inplace = True) 

In [103]:
df['Dep_Hour'] = df['Dep_Time'].str.split(':').str[0]
df['Dep_Minutes'] = df['Dep_Time'].str.split(':').str[1]

In [104]:
df['Dep_Hour'] = df['Dep_Hour'].astype(int)
df['Dep_Minutes'] = df['Dep_Minutes'].astype(int)


In [105]:
df.drop('Dep_Time',axis = 1 , inplace = True) 

In [106]:
df['Duration_hour'] = df['Duration'].str.split(' ').str[0].str[0]
df['Duration_minutes'] = df['Duration'].str.split(' ').str[0].str[0]



In [109]:
df['Duration_hour'] = df['Duration_hour'].astype(int)
df['Duration_minutes'] = df['Duration_minutes'].astype(int)


In [110]:
df.drop(['Duration'] , axis = 1 , inplace = True ) 

In [115]:
df['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', '4 stops'],
      dtype=object)

In [118]:
df['Total_Stops'] = df['Total_Stops'].map({'non-stop' : 0 , '2 stops' : 2 , '1 stop' : 1 , '3 stops' : 3 , '4 stops' : 4} )

df['Total_Stops'] = df['Total_Stops'].astype(int)

In [123]:
df.drop('Route' , axis = 1 , inplace = True) 


In [124]:
df.dtypes

Airline              object
Source               object
Destination          object
Total_Stops           int32
Additional_Info      object
Price               float64
Date                  int32
Month                 int32
Year                  int32
Arrival_Hour          int32
Arrival_Minutes       int32
Dep_Hour              int32
Dep_Minutes           int32
Duration_hour         int32
Duration_minutes      int32
dtype: object

In [130]:
df1 = pd.get_dummies(df , columns = ['Airline' , 'Source' , 'Destination' , 'Additional_Info'] , drop_first = True)

In [134]:
features = df1.drop('Price' , axis = 1)
label = df1['Price']